In [1]:
from hstar import process
from hstar import trilinear
from hzz import angles
from hzz import zcandidate

import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
sample = process.Sample(weight='wt', 
    amplitude = process.Basis.SBI, components = {
    process.Basis.SBI: 'msq_sbi_sm',
    process.Basis.SIG: 'msq_sig_sm',
    process.Basis.BKG: 'msq_bkg_sm',
    process.Basis.INT: 'msq_int_sm'
  })

sample.open(csv = [
  '../ggZZ2e2m_all_new.csv',
  '../ggZZ4e_all_new.csv',
  '../ggZZ4m_all_new.csv'
  ], xs=[1.4783394, 0.47412769, 0.47412769],
    lumi=3000., k=1.83, nrows=1000000
)

In [3]:
zmasses = zcandidate.ZmassPairChooser(sample)
leptons = zmasses.find_Z()

kinematics, legal_inds = angles.calculate(leptons[:,0], leptons[:,1], leptons[:,2], leptons[:,3])

kinematics = kinematics[legal_inds]
zmasses.df = zmasses.df.take(legal_inds)
zmasses.sample.events = zmasses.df

leptons = leptons[legal_inds]

c6_mod = trilinear.Modifier(c6_values = [-5,-1,0,1,5], c6_amplitudes = ['msq_sbi_c6_6', 'msq_sbi_c6_10', 'msq_sbi_c6_11', 'msq_sbi_c6_12', 'msq_sbi_c6_16'])
c6_weights = c6_mod.modify(sample=zmasses.sample, c6=[-20,-10,10,20,30])

Deleting refs p12, p23, p14, p34
Deleting ref chi_sq
Deleting ref cond
Deleting refs pairs, lepton_pairs, closest_pair_indices, closest_pair, pT_max_ind, pT_min_ind
0


ValueError: too many values to unpack (expected 2)

In [ ]:
m4l = (leptons[:,0] + leptons[:,1] + leptons[:,2] + leptons[:,3]).mass
m4l = zmasses.m4l[legal_inds]

hist, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=zmasses.sample.events['wt'])
hist1, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=c6_weights.T[1]) # -10
hist2, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=c6_weights.T[2]) # 10
hist3, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=c6_weights.T[0]) # -20
hist4, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=c6_weights.T[3]) # 20
hist5, bins = np.histogram(m4l, bins=60, range=(180,1000), weights=c6_weights.T[4]) # 30

fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={'height_ratios': [4, 1]}, figsize=(6,7))

ax1.set_xticklabels([])

ax1.step(bins[:-1], hist, color='k', label=u'SM ($c_6=0$)', linestyle='--')
ax1.step(bins[:-1], hist1, color='r', label=u'BSM ($c_6=-10$)')
ax1.step(bins[:-1], hist2, color='b', label=u'BSM ($c_6=10$)')
#ax1.step(bins[:-1], hist5, color='b', label=u'BSM ($c_6=30$)')

ax1.set_ylabel('Number of events []')
ax1.set_yscale('log')

ax1.legend()


ax2.step(bins[:-1], hist/hist, color='k', linestyle='--', label=u'SM ($c_6=0$)')
ax2.step(bins[:-1], hist1/hist, color='r', label=u'BSM ($c_6=-10$)')
ax2.step(bins[:-1], hist2/hist, color='b', label=u'BSM ($c_6=10$)')
#ax2.step(bins[:-1], hist5/hist, color='b', label=u'BSM ($c_6=30$)')

ax2.set_xlabel(u'$m_{4\ell}$ [GeV]')
ax2.set_ylabel('R []')
ax2.set_ybound(0.99,1.02)

fig.tight_layout()

plt.show()